# Тестовое задание в БД Nebula

1. Можно через Docker Desktop установить Nebula Extension.
2. По инструкции запустить сервер.
3. Создать свое пространство (SPACE), по аналогии базы данных в MySQL.
4. Чтобы пакетно импортировать данные в NebulaGraph, у вас должна быть схема графа. Вы можете создать схему на странице консоли или на странице схемы Studio.

Чтобы создать схему графиков в Studio, необходимо проверить:

- Студия подключена к NebulaGraph.
- Ваша учетная запись имеет привилегию GOD, ADMIN или DBA.
- Схема разработана.
- Создается пространство графа.

В пространстве (SPACE) можно задать тип `id` узлов. Зададим `id`, имеющий тип `STRING` и размер `60`. Все наши узлы имеют уникальный `id`, значит, не надо будет вводить дополнительные ограничения на уникальность имени участника и события.

Создание схемы:

- cоздание тегов (лейблов);
- создание типов ребер.

Это все делается перед тем, как пишутся запросы на `Cypher`.

Чтение исходного файла представляется невозможным, поэтому создадим небольшой скрипт, меняющий формат исходного файла на формат, который поддерживает `Nebula`.

## Формат файла, описывающего вершины

### :Participant

_participant_with_header.csv_

```
:VID(string)
Галчевская Карина Владимировна
Белоновская Анастасия Семеновна
Офицеров Олег Романович
```

### :Event

_events_with_header.csv_

```
:VID(string)
189
206
445
```

## Формат файла, описывающего связи

### WITH

_WITH_with_header.csv_

```
:SRC_VID(string),:DST_VID(string)
Галчевская Карина Владимировна,Белоновская Анастасия Семеновна
Офицеров Олег Романович,Сапожник Борис Валерьевич
Жандарова Лариса Германовна,Чемодуров Дамир Русланович
```

### HAS_BEEN

_HAS_BEEN_with_header.csv_

```
:SRC_VID(string),:DST_VID(string)
Галчевская Карина Владимировна,189
Офицеров Олег Романович,206
Жандарова Лариса Германовна,445
```


## Импортируем библиотеки

In [1]:
import numpy as np
import pandas as pd

## Считываем исходный файл

In [2]:
df = pd.read_csv("data/data_test_com.csv")

In [3]:
df

,id события,ФИО участника события 1,ФИО участника события 2
0,189,Галчевская Карина Владимировна,Белоновская Анастасия Семеновна
1,206,Офицеров Олег Романович,Сапожник Борис Валерьевич
2,445,Жандарова Лариса Германовна,Чемодуров Дамир Русланович
3,503,Масимова Яна Дамировна,Мингажетдинов Рамиль Семенович
4,571,Мухтарова Алена Яковлевна,Щербатенко Ольга Робертовна
...,...,...,...
4995,999333,Осташов Владимир Данилович,Чалов Илья Владимирович
4996,999360,Гандыбина Любовь Александровна,Мерлин Илья Юрьевич
4997,999403,Востоков Виктор Ильдарович,Аликас Никита Андреевич
4998,999405,Огарева Людмила Ильдаровна,Нагайцева Алина Степановна


In [73]:
df1 = pd.DataFrame(list(df['ФИО участника события 1']), columns=[':VID(string)'])
df2 = pd.DataFrame(list(df['ФИО участника события 2']), columns=[':VID(string)'])
df_event = pd.DataFrame(list(df['id события']), columns=[':VID(string)'])

dict1 = {':SRC_VID(string)': list(df['ФИО участника события 1']), ':DST_VID(string)': list(df['id события'])}
dict2 = {':SRC_VID(string)': list(df['ФИО участника события 2']), ':DST_VID(string)': list(df['id события'])}
df1_1 = pd.DataFrame(dict1)
df2_1 = pd.DataFrame(dict2)

## Создаем 4 файла с вершинами и рёбрами графа

In [72]:
df_participant = pd.concat([df1, df2]).drop_duplicates()
df_has_been = pd.concat([df1_1, df2_1], ignore_index=True).drop_duplicates()

df_participant.to_csv('data/participant_with_header.csv', index=False)  
df_event.to_csv('data/event_with_header.csv', index=False) 

# ':SRC_VID(string)',':DST_VID(string)'
df[['ФИО участника события 1', 'ФИО участника события 2']].to_csv('data/WITH_with_header.csv', index=False, header=[':SRC_VID(string)',':DST_VID(string)'])
df_has_been.to_csv('data/HAS_BEEN_with_header.csv', index=False) 

## Импорт данных

Импорт данных производится не на `Cypher`, а в консоли nebula или в NebulaGraph Studio.

![import](img/imprt.png)

Запрос на отображение вершин `:Participant`:

```
MATCH(n:Participant)
RETURN n
LIMIT 1000
```

Результат выполнения запроса:

![part](img/cypher.png)

Запрос на отображение вершин `:Event`:

```
MATCH(n:Event) 
RETURN n 
LIMIT 1000
```

Запрос на отображение связей вершины с определенным `id`:

```
MATCH (s)-[e]->() 
WHERE id(s) == "Галчевская Карина Владимировна" 
RETURN e;
```

Результат выполнения запроса:

![gal](img/gal.png)